In [1]:
import pandas as pd
import numpy as np
import json 
import time 
import requests
import pycountry

ModuleNotFoundError: No module named 'pandas'

# IMF Example

A quick example of requesting and transforming data. I'll use the IMF API with just one series.

</br> </br>

The IMF api is decribed [here]("https://datahelp.imf.org/knowledgebase/articles/1968408-how-to-use-the-api-python-and-r).  Queries are made up of

"""
    
    Following that root, you will need to add on additional details about what data you would like:

    Data type: "CompactData" to retrieve data, "DataStructure" to retrieve series information, or "GenericMetadata" to get metadata. 
    Database you would like to query: For example, International Financial Statistics ("IFS"), Balance of Payments ("BOP"), etc. 
    Frequency: Annual ("A"), Quarterly ("Q"), Monthly ("M")
    Specific Indicator interested in: For CPI this is " PCPI_IX"
    So, if you want to query the IFS database for monthly CPI data for 2000-2001, you would want to use the URL:
"""

I'll use their example of requesting monthly CPI data but customize it a bit - include newer data.


In [2]:
url = "http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/M..PCPI_IX.?startPeriod=2020&endPeriod=2021"
req = requests.get(url)

req


<Response [200]>

Let's look at the returned data.

In [3]:
req.json()

{'CompactData': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema',
  '@xsi:schemaLocation': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd http://dataservices.imf.org/compact/IFS http://dataservices.imf.org/compact/IFS.xsd',
  '@xmlns': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message',
  'Header': {'ID': '07247772-8bbc-4569-ad71-96d242441939',
   'Test': 'false',
   'Prepared': '2024-09-09T09:02:56',
   'Sender': {'@id': '1C0',
    'Name': {'@xml:lang': 'en', '#text': 'IMF'},
    'Contact': {'URI': 'http://www.imf.org',
     'Telephone': '+ 1 (202) 623-6220'}},
   'Receiver': {'@id': 'ZZZ'},
   'DataSetID': 'IFS'},
  'DataSet': {'@xmlns': 'http://dataservices.imf.org/compact/IFS',
   'Series': [{'@FREQ': 'M',
     '@REF_AREA': 'BH',
     '@INDICATOR': 'PCPI_IX',
     '@UNIT_MULT': '0',
     '@BASE_YEAR': '2010=100',
     '@TIME_FORMAT': 'P1M',
     

It looks like there are lots of different `Series`, one for each country

In [24]:
data = req.json()
series = data['CompactData']['DataSet']['Series']
series[:2] # print the first 2 series

[{'@FREQ': 'M',
  '@REF_AREA': 'BH',
  '@INDICATOR': 'PCPI_IX',
  '@UNIT_MULT': '0',
  '@BASE_YEAR': '2010=100',
  '@TIME_FORMAT': 'P1M',
  'Obs': [{'@TIME_PERIOD': '2020-01', '@OBS_VALUE': '118.179204656161'},
   {'@TIME_PERIOD': '2020-02', '@OBS_VALUE': '118.416989776194'},
   {'@TIME_PERIOD': '2020-03', '@OBS_VALUE': '117.109171616015'},
   {'@TIME_PERIOD': '2020-04', '@OBS_VALUE': '114.612427855673'},
   {'@TIME_PERIOD': '2020-05', '@OBS_VALUE': '115.325783215771'},
   {'@TIME_PERIOD': '2020-06', '@OBS_VALUE': '114.612427855673'},
   {'@TIME_PERIOD': '2020-07', '@OBS_VALUE': '115.087998095738'},
   {'@TIME_PERIOD': '2020-08', '@OBS_VALUE': '115.444675775787'},
   {'@TIME_PERIOD': '2020-09', '@OBS_VALUE': '116.990279055999'},
   {'@TIME_PERIOD': '2020-10', '@OBS_VALUE': '115.206890655755'},
   {'@TIME_PERIOD': '2020-11', '@OBS_VALUE': '114.969105535722'},
   {'@TIME_PERIOD': '2020-12', '@OBS_VALUE': '116.276923695901'},
   {'@TIME_PERIOD': '2021-01', '@OBS_VALUE': '114.969105535722'

Let's reshape this data. Right now we have dicts with top level parameters:

`'@FREQ': 'M',
  '@REF_AREA': 'BH',
  '@INDICATOR': 'PCPI_IX',
  '@UNIT_MULT': '0',
  '@BASE_YEAR': '2010=100',
  '@TIME_FORMAT': 'P1M',`

and then `Obs` with the values. Let's take the `REF_AREA` into the observations and merge all the data.


In [37]:
parsed_data = [] # create an empty list to store the parsed data

for country_series in series: # loop over all the sets of data
    country = country_series['@REF_AREA'] # get the country code
    for obs in country_series.get('Obs', []):
        date = obs['@TIME_PERIOD']+"-01" # get the date
        value = obs['@OBS_VALUE']
        iso2 = country_series['@REF_AREA']
        parsed_data.append({'date': date, 'value': value, 'iso2': iso2})


Now make a df of it, and add the iso3s using pycountry

In [44]:
df = pd.DataFrame(parsed_data) # create a DataFrame from the parsed data

def get_iso3(iso2):
    try:
        return pycountry.countries.get(alpha_2=iso2).alpha_3
    except:
        print(f"Error with {iso2}")
        return np.nan
    
df['id'] = df['iso2'].apply(lambda x: get_iso3(x))
df = df.dropna()
df = df.drop(columns=['iso2'])

# and let's add the common name of the country
df['name'] = df['id'].apply(lambda x: pycountry.countries.get(alpha_3=x).name)
df

Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK
Error with XK


,date,value,id,name
0,2020-01-01,118.179204656161,BHR,Bahrain
1,2020-02-01,118.416989776194,BHR,Bahrain
2,2020-03-01,117.109171616015,BHR,Bahrain
3,2020-04-01,114.612427855673,BHR,Bahrain
4,2020-05-01,115.325783215771,BHR,Bahrain
...,...,...,...,...
3961,2021-08-01,354.540499251802,UZB,Uzbekistan
3962,2021-09-01,358.404990693647,UZB,Uzbekistan
3963,2021-10-01,363.100096071733,UZB,Uzbekistan
3964,2021-11-01,367.529917243809,UZB,Uzbekistan


In [28]:
country_series

{'@FREQ': 'M',
 '@REF_AREA': 'CSH',
 '@INDICATOR': 'PCPI_IX',
 '@UNIT_MULT': '0',
 '@TIME_FORMAT': 'P1M'}